# Comparison Statistics between Cohorts

## Python Setup

In [61]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas import Series
import glob
from openpyxl import load_workbook
import re
pd.options.display.max_columns = 100

## Load In Data

### Original Data

In [62]:
inca_funded_grnts = pd.read_csv('../data/inca_funded_grants.csv', low_memory=False)
inca_grnts = pd.read_csv('../data/inca_researcher_grants.csv', low_memory=False)
inca_pubs = pd.read_csv('../data/inca_researcher_publications.csv', low_memory=False)

In [63]:
print(inca_funded_grnts[inca_funded_grnts['rsr_id'].isnull()].shape)
print(inca_funded_grnts[inca_funded_grnts['grant_id'].isnull()].shape)
print(inca_grnts[inca_grnts['rsr_id'].isnull()].shape)
print(inca_grnts[inca_grnts['grant_id'].isnull()].shape)
print(inca_pubs[inca_pubs['rsr_id'].isnull()].shape)
print(inca_pubs[inca_pubs['pub_id'].isnull()].shape)

(0, 6)
(0, 6)
(0, 7)
(0, 7)
(158218, 11)
(0, 11)


In [64]:
# inca_funded_grnts = inca_funded_grnts[(inca_funded_grnts['rsr_id'].notnull())
#                                       &(inca_funded_grnts['grant_id'].notnull())].reset_index(drop=True)
# inca_grnts = inca_grnts[(inca_grnts['rsr_id'].notnull())
#                         &(inca_grnts['grant_id'].notnull())].reset_index(drop=True)
# inca_pubs = inca_pubs[(inca_pubs['rsr_id'].notnull())
#                       &(inca_pubs['pub_id'].notnull())].reset_index(drop=True)

### Counterfactual Data

In [65]:
funded_grnts = pd.read_csv('../data/counterfactual_funded_grants.csv', low_memory=False)
grnts = pd.read_csv('../data/counterfactual_researcher_grants.csv', low_memory=False)
pubs = pd.read_csv('../data/counterfactual_researcher_publications.csv', low_memory=False)

In [66]:
print(funded_grnts[funded_grnts['rsr_id'].isnull()].shape)
print(funded_grnts[funded_grnts['grant_id'].isnull()].shape)
print(grnts[grnts['rsr_id'].isnull()].shape)
print(grnts[grnts['grant_id'].isnull()].shape)
print(pubs[pubs['rsr_id'].isnull()].shape)
print(pubs[pubs['pub_id'].isnull()].shape)

(0, 7)
(0, 7)
(0, 7)
(0, 7)
(451, 11)
(0, 11)


In [67]:
# funded_grnts = funded_grnts[(funded_grnts['rsr_id'].notnull())
#                             &(funded_grnts['grant_id'].notnull())].reset_index(drop=True)
# grnts = grnts[(grnts['rsr_id'].notnull())
#               &(grnts['grant_id'].notnull())].reset_index(drop=True)
# pubs = pubs[(pubs['rsr_id'].notnull())
#             &(pubs['pub_id'].notnull())].reset_index(drop=True)

## Combining Data

In [68]:
funded_grnts = pd.concat([funded_grnts, inca_funded_grnts]).drop_duplicates().reset_index(drop = True)
grnts = pd.concat([grnts, inca_grnts]).drop_duplicates().reset_index(drop = True)
pubs = pd.concat([pubs, inca_pubs]).drop_duplicates().reset_index(drop = True)

In [69]:
funded_grnts = funded_grnts.drop_duplicates(['grant_id', 'rsr_id', 'funder_name']).reset_index(drop=True)
grnts = grnts.drop_duplicates(['grant_id', 'rsr_id', 'funder_name']).reset_index(drop=True)
pubs = pubs.drop_duplicates(['pub_id', 'rsr_id']).reset_index(drop=True)

In [70]:
funders = sorted(list(funded_grnts['funder_name'].unique()))
print(funders)

['Cancer Research UK', 'INCa/INSERM/DGOS', 'INCa/INSERM/DGOS - ORCID Confirmed', 'National Cancer Institute', 'National Health and Medical Research Council', 'Wellcome Trust']


## Cleaning Data

In [71]:
funded_grnts['start_date'] = pd.to_datetime(funded_grnts['start_date'])
funded_grnts['end_date'] = pd.to_datetime(funded_grnts['end_date'])
grnts['start_date'] = pd.to_datetime(grnts['start_date'])
grnts['end_date'] = pd.to_datetime(grnts['end_date'])
pubs['date'] = pd.to_datetime(pubs['date'])

In [72]:
funded_grnts['funding_len'] = (funded_grnts['end_date']-funded_grnts['start_date'])/timedelta(days=365)
grnts['funding_len'] = (grnts['end_date']-grnts['start_date'])/timedelta(days=365)

In [73]:
funded_grnts['nb_rsrs'] = funded_grnts.groupby('grant_id')['rsr_id'].transform('nunique')
funded_grnts['nb_rsrs'] = funded_grnts['nb_rsrs'].replace(0, np.nan)
grnts['nb_rsrs'] = grnts.groupby('grant_id')['rsr_id'].transform('nunique')
grnts['nb_rsrs'] = grnts['nb_rsrs'].replace(0, np.nan)
pubs['nb_rsrs'] = pubs.groupby('pub_id')['rsr_id'].transform('nunique')
pubs['nb_rsrs'] = pubs['nb_rsrs'].replace(0, np.nan)

In [74]:
funded_grnts['start_year'] = pd.DatetimeIndex(funded_grnts['start_date']).year
grnts['start_year'] = pd.DatetimeIndex(grnts['start_date']).year
pubs['year'] = pd.DatetimeIndex(pubs['date']).year

### Create Lookup Tables

In [75]:
funded_grnts_funder = funded_grnts[['funder_name', 'grant_id']].drop_duplicates().reset_index(drop=True)
grnts_funder = grnts[['funder_name', 'grant_id']].drop_duplicates().reset_index(drop=True)

In [76]:
funded_grnts_rsr = funded_grnts[['rsr_id', 'grant_id']].drop_duplicates().reset_index(drop=True)
grnts_rsr = grnts[['rsr_id', 'grant_id']].drop_duplicates().reset_index(drop=True)
pubs_rsr = pubs[['rsr_id', 'rsr_country', 'rsr_affiliation', 'rsr_affiliation_id', 'pub_id']].drop_duplicates().reset_index(drop=True)

In [77]:
to_remove = ('rsr_id', 'funder_id', 'funder_name', 'rcdc_name', 'rcdc_code')
cols = [col for col in list(funded_grnts) if col not in to_remove]
funded_grnts_info = funded_grnts[cols].drop_duplicates().reset_index(drop=True)

In [78]:
to_remove = ('rsr_id', 'funder_id', 'funder_name', 'rcdc_name', 'rcdc_code')
cols = [col for col in list(grnts) if col not in to_remove]
grnts_info = grnts[cols].drop_duplicates('grant_id').reset_index(drop=True)

In [79]:
to_remove = ('rsr_id', 'rsr_country', 'rsr_affiliation', 'rsr_affiliation_id')
cols = [col for col in list(pubs) if col not in to_remove]
pubs_info = pubs[cols].drop_duplicates('pub_id').reset_index(drop=True)

## Import Topic Lookup Tables

In [80]:
files = glob.glob('../data/topic_lookups/*_grants_rcdc.csv')
grnts_rcdc = pd.concat([pd.read_csv(f, low_memory=False) for f in files])
grnts_rcdc = grnts_rcdc.drop_duplicates().sort_values('grant_id').reset_index(drop=True)
grnts_rcdc['nb_rcdc'] = grnts_rcdc.groupby('grant_id')['grant_id'].transform('count')

In [81]:
files = glob.glob('../data/topic_lookups/*_publications_rcdc.csv')
pubs_rcdc = pd.concat([pd.read_csv(f, low_memory=False) for f in files])
pubs_rcdc = pubs_rcdc.drop_duplicates().sort_values('pub_id').reset_index(drop=True)
pubs_rcdc['nb_rcdc'] = pubs_rcdc.groupby('pub_id')['pub_id'].transform('count')

In [82]:
files = glob.glob('../data/topic_lookups/*_publications_cso.csv')
pubs_cso = pd.concat([pd.read_csv(f, low_memory=False) for f in files])
pubs_cso = pubs_cso.drop_duplicates().sort_values('pub_id').reset_index(drop=True)
pubs_cso['nb_cso'] = pubs_cso.groupby('pub_id')['pub_id'].transform('count')

## Merging on Prior and Subsequent Grants and Publications

I keep only grants and publications that are within 5 years of the funded grant.

In [83]:
start_year=2007
end_year=2012

col1=[]
col2=[]
col3=[]   
for grant_year in range(start_year, end_year+1):
    for award_year in range(grant_year-5, grant_year+6):
        if grant_year>award_year:
            col1.append(grant_year)
            col2.append("pre")
            col3.append(award_year)
        if grant_year==award_year:
            col1.append(grant_year)
            col2.append("focal")
            col3.append(award_year)
        if grant_year<award_year:
            col1.append(grant_year)
            col2.append("post")
            col3.append(award_year)
merge_key = pd.DataFrame({'start_year':col1, 'status':col2, 'year':col3})

In [84]:
temp = pd.merge(funded_grnts[['funder_name', 'grant_id', 'start_year', 'start_date', 'rsr_id']].drop_duplicates(), 
                merge_key, how='left', on='start_year')

In [85]:
grnt_cols = ['rsr_id', 'grant_id', 'start_year']
grnts_mrg = pd.merge(temp, grnts[grnt_cols].drop_duplicates().rename(columns={'start_year':'year'}),
                     how='left', on=['rsr_id', 'year'], suffixes=('', '_2'))

In [86]:
pub_cols = ['rsr_id', 'pub_id', 'year']
pubs_mrg = pd.merge(temp, pubs[pub_cols].drop_duplicates(),
                    how='left', on=['rsr_id', 'year'], suffixes=('', '_2'))

## 1. Funded Grants Statistics

### Number of Grants per Funder

In [87]:
funded_grnts.describe(include='all')

,end_date,funder_id,funder_name,funding_amount,grant_id,rsr_id,start_date,funding_len,nb_rsrs,start_year
count,21954,20501,22002,1.970700e+04,22002,22002,22002,21954.000000,22002.000000,22002.000000
unique,1081,4,6,NaN,16926,14005,1171,NaN,NaN,NaN
top,2017-01-01 00:00:00,grid.48336.3a,National Cancer Institute,NaN,grant.2438876,ur.01117731572.33,2009-01-01 00:00:00,NaN,NaN,NaN
freq,828,14682,14682,NaN,45,384,1613,NaN,NaN,NaN
first,2007-01-31 00:00:00,NaN,NaN,NaN,NaN,NaN,2007-01-01 00:00:00,NaN,NaN,NaN
last,2024-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,2012-12-31 00:00:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,2.349073e+06,NaN,NaN,NaN,4.239022,2.275520,2009.450368
std,NaN,NaN,NaN,9.219695e+06,NaN,NaN,NaN,2.658188,3.377899,1.661127
min,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.002740,1.000000,2007.000000
25%,NaN,NaN,NaN,3.206820e+05,NaN,NaN,NaN,2.002740,1.000000,2008.000000


In [88]:
print("Number of unique researchers funded:")
print(funded_grnts['rsr_id'].nunique())
funded_grnts_unique_rsrs_tot = funded_grnts['rsr_id'].nunique()

Number of unique researchers funded:
14005


In [89]:
print("Number of unique researchers funded by each agency:\n")
print(funded_grnts.groupby('funder_name')['rsr_id'].nunique())
funded_grnts_unique_rsrs = list(funded_grnts.groupby('funder_name')['rsr_id'].nunique())

Number of unique researchers funded by each agency:

funder_name
Cancer Research UK                              1419
INCa/INSERM/DGOS                                 811
INCa/INSERM/DGOS - ORCID Confirmed               170
National Cancer Institute                       9356
National Health and Medical Research Council    2015
Wellcome Trust                                   330
Name: rsr_id, dtype: int64


In [90]:
print("Number of unique grants funded:")
print(funded_grnts['grant_id'].nunique())
funded_grnts_unique_grnts_tot = funded_grnts['grant_id'].nunique()

Number of unique grants funded:
16926


In [91]:
print("Number of unique grants funded by each agency:\n")
print(funded_grnts.groupby('funder_name')['grant_id'].nunique())
funded_grnts_unique_grnts = list(funded_grnts.groupby('funder_name')['grant_id'].nunique())

Number of unique grants funded by each agency:

funder_name
Cancer Research UK                               2254
INCa/INSERM/DGOS                                 1104
INCa/INSERM/DGOS - ORCID Confirmed                273
National Cancer Institute                       11936
National Health and Medical Research Council     1155
Wellcome Trust                                    263
Name: grant_id, dtype: int64


### Number of Grants per Researcher

In [92]:
print("Agerage Number of Grants per Researcher:")
print(funded_grnts.groupby('rsr_id')['grant_id'].nunique().mean())
funded_grnts_grnts_per_rsr = [funded_grnts.groupby('rsr_id')['grant_id'].nunique().mean()
                              , funded_grnts.groupby('rsr_id')['grant_id'].nunique().std()]
print("\nNumber of grants from agencies per researcher (as % of total):\n")
print(funded_grnts.groupby('rsr_id')['grant_id'].nunique().value_counts(normalize=True).head())

Agerage Number of Grants per Researcher:
1.5710103534451982

Number of grants from agencies per researcher (as % of total):

1    0.715245
2    0.168797
3    0.059122
4    0.026205
5    0.013995
Name: grant_id, dtype: float64


In [93]:
print("Agerage Number of Agencies per Researcher:")
print(funded_grnts.groupby('rsr_id')['funder_name'].nunique().mean())
funded_grnts_agencies_per_rsr_tot = [funded_grnts.groupby('rsr_id')['funder_name'].nunique().mean(),
                                      funded_grnts.groupby('rsr_id')['funder_name'].nunique().std()]
print("\nBy how many agencies are the researchers funded?\n")
print(funded_grnts.groupby('rsr_id')['funder_name'].nunique().value_counts(normalize=True))

Agerage Number of Agencies per Researcher:
1.0068546947518744

By how many agencies are the researchers funded?

1    0.993574
2    0.006069
3    0.000286
4    0.000071
Name: funder_name, dtype: float64


In [94]:
print("How many grants does each agency give to its reserachers in the 5 focal years?")
for funder in funders:
    print("\n{}:".format(funder))
    temp = funded_grnts[funded_grnts['funder_name']==funder].groupby('rsr_id')['grant_id'].nunique().reset_index()
    print(temp['grant_id'].value_counts(normalize=True).head(3))
    print("(Total researchers: {})".format(temp.shape[0]))

How many grants does each agency give to its reserachers in the 5 focal years?

Cancer Research UK:
1    0.718816
2    0.156448
3    0.049331
Name: grant_id, dtype: float64
(Total researchers: 1419)

INCa/INSERM/DGOS:
1    0.723798
2    0.157830
3    0.060419
Name: grant_id, dtype: float64
(Total researchers: 811)

INCa/INSERM/DGOS - ORCID Confirmed:
1    0.652941
2    0.200000
4    0.064706
Name: grant_id, dtype: float64
(Total researchers: 170)

National Cancer Institute:
1    0.716332
2    0.173792
3    0.058893
Name: grant_id, dtype: float64
(Total researchers: 9356)

National Health and Medical Research Council:
1    0.714640
2    0.155335
3    0.065509
Name: grant_id, dtype: float64
(Total researchers: 2015)

Wellcome Trust:
1    0.803030
2    0.160606
4    0.021212
Name: grant_id, dtype: float64
(Total researchers: 330)


### Grant Characteristics

In [95]:
df = pd.merge(funded_grnts_funder, funded_grnts_info, how='left', on='grant_id')
print("Average Grant Amount: {}".format(df['funding_amount'].mean()))
funded_grnts_amt_tot = [df['funding_amount'].mean(), df['funding_amount'].std()]
print("Average Funding Length: {}".format(df['funding_len'].mean()))
funded_grnts_len_tot = [df['funding_len'].mean(), df['funding_len'].std()]
print("Average Team Size: {}".format(df['nb_rsrs'].mean()))
funded_grnts_team_size_tot = [df['nb_rsrs'].mean(), df['nb_rsrs'].std()]

funded_grnts_amt_avg = []
funded_grnts_amt_med = []
funded_grnts_len_avg = []
funded_grnts_len_med = []
funded_grnts_team_size_avg = []
funded_grnts_team_size_med = []
for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder].copy()
    print(temp[['funding_amount', 'funding_len', 'nb_rsrs']].describe())
    print("(Total grants: {})".format(temp.shape[0]))
    funded_grnts_amt_avg.append(temp['funding_amount'].mean())
    funded_grnts_amt_med.append(temp['funding_amount'].median())
    funded_grnts_len_avg.append(temp['funding_len'].mean())
    funded_grnts_len_med.append(temp['funding_len'].median())
    funded_grnts_team_size_avg.append(temp['nb_rsrs'].mean())
    funded_grnts_team_size_med.append(temp['nb_rsrs'].median())

Average Grant Amount: 1654072.905751341
Average Funding Length: 4.015720316673971
Average Team Size: 1.3026788342655284

Cancer Research UK:
       funding_amount  funding_len      nb_rsrs
count             0.0  2254.000000  2254.000000
mean              NaN     3.372958     1.016415
std               NaN     2.070945     0.127094
min               NaN     0.002740     1.000000
25%               NaN     1.997260     1.000000
50%               NaN     3.000000     1.000000
75%               NaN     5.000000     1.000000
max               NaN    13.008219     2.000000
(Total grants: 2254)

INCa/INSERM/DGOS:
       funding_amount  funding_len      nb_rsrs
count    1.103000e+03  1100.000000  1104.000000
mean     4.846043e+05     2.827345     1.163949
std      8.546769e+05     0.812899     0.394115
min      2.028000e+04     0.509589     1.000000
25%      1.715210e+05     2.169863     1.000000
50%      3.657100e+05     3.000000     1.000000
75%      5.953145e+05     3.002740     1.000000
max

In [96]:
temp = pd.merge(funded_grnts_funder, funded_grnts_info, how='left', on='grant_id')
temp['year'] = pd.DatetimeIndex(temp['start_date']).year
pd.crosstab(temp['funder_name'], temp['year'])

year,2007,2008,2009,2010,2011,2012
funder_name,,,,,,
Cancer Research UK,549,428,346,314,346,271
INCa/INSERM/DGOS,239,121,165,162,205,212
INCa/INSERM/DGOS - ORCID Confirmed,55,35,35,42,57,49
National Cancer Institute,1740,2311,2799,1739,1721,1626
National Health and Medical Research Council,141,208,198,132,243,233
Wellcome Trust,33,40,37,54,55,44


### RCDC Codes Analysis

In [97]:
df = pd.merge(funded_grnts_info, grnts_rcdc, how='left', on='grant_id')

In [98]:
print("Most Common RCDC Codes:")
print(df['rcdc_name'].value_counts().head())
funded_grnts_rcdc_1_tot = df['rcdc_name'].value_counts().index[0]
funded_grnts_rcdc_2_tot = df['rcdc_name'].value_counts().index[1]
funded_grnts_rcdc_3_tot = df['rcdc_name'].value_counts().index[2]
funded_grnts_rcdc_4_tot = df['rcdc_name'].value_counts().index[3]
funded_grnts_rcdc_5_tot = df['rcdc_name'].value_counts().index[4]

Most Common RCDC Codes:
Cancer               13505
Clinical Research     5645
Genetics              5149
Rare Diseases         4890
Biotechnology         4452
Name: rcdc_name, dtype: int64


In [99]:
df = pd.merge(funded_grnts_funder, grnts_rcdc, how='left', on='grant_id')

In [100]:
print("Average number of RCDC's per Grant:\n")
print(df[['funder_name', 'grant_id', 'nb_rcdc']].drop_duplicates().groupby('funder_name')['nb_rcdc'].mean())

Average number of RCDC's per Grant:

funder_name
Cancer Research UK                              3.351375
INCa/INSERM/DGOS                                5.540863
INCa/INSERM/DGOS - ORCID Confirmed              5.533582
National Cancer Institute                       5.611595
National Health and Medical Research Council    4.793074
Wellcome Trust                                  4.498099
Name: nb_rcdc, dtype: float64


In [101]:
funded_grnts_rcdc_1 = []
funded_grnts_rcdc_2 = []
funded_grnts_rcdc_3 = []
funded_grnts_rcdc_4 = []
funded_grnts_rcdc_5 = []

for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder]
    print(temp['rcdc_name'].value_counts().head())
    funded_grnts_rcdc_1.append(temp['rcdc_name'].value_counts().index[0])
    funded_grnts_rcdc_2.append(temp['rcdc_name'].value_counts().index[1])
    funded_grnts_rcdc_3.append(temp['rcdc_name'].value_counts().index[2])
    funded_grnts_rcdc_4.append(temp['rcdc_name'].value_counts().index[3])
    funded_grnts_rcdc_5.append(temp['rcdc_name'].value_counts().index[4])


Cancer Research UK:
Cancer               1440
Clinical Research     636
Genetics              581
Rare Diseases         484
Biotechnology         322
Name: rcdc_name, dtype: int64

INCa/INSERM/DGOS:
CANCER               1009
CLINICAL RESEARCH     552
RARE DISEASES         498
GENETICS              345
BIOTECHNOLOGY         250
Name: rcdc_name, dtype: int64

INCa/INSERM/DGOS - ORCID Confirmed:
CANCER               248
CLINICAL RESEARCH    138
RARE DISEASES        115
GENETICS              91
BIOTECHNOLOGY         63
Name: rcdc_name, dtype: int64

National Cancer Institute:
Cancer               10635
Clinical Research     4714
Genetics              4161
Rare Diseases         3941
Biotechnology         3924
Name: rcdc_name, dtype: int64

National Health and Medical Research Council:
Cancer               1155
Rare Diseases         364
Genetics              300
Prevention            271
Clinical Research     244
Name: rcdc_name, dtype: int64

Wellcome Trust:
Cancer                263
Genet

## 2. Prior and Subsequent Grants

In [102]:
print("Average Funding Length: {}".format(grnts_info['funding_len'].mean()))
grnts_len_tot = [grnts_info['funding_len'].mean(), grnts_info['funding_len'].std()]
print("Average Team Size: {}".format(grnts_info['nb_rsrs'].mean()))
grnts_team_size_tot = [grnts_info['nb_rsrs'].mean(), grnts_info['nb_rsrs'].std()]
print("Average Funding Amount: {}".format(grnts_info['funding_amount'].mean()))
grnts_amt_tot = [grnts_info['funding_amount'].mean(), grnts_info['funding_amount'].std()]

Average Funding Length: 4.626028372686579
Average Team Size: 2.3963329412578704
Average Funding Amount: 1727084.334653025


In [103]:
def grant_groupby(grnts_mrg, yearly=False):
    
    grnts_info.columns = [col+"_2" for col in list(grnts_info)]
    df = pd.merge(grnts_mrg, grnts_info.rename(columns={'start_year_2':'year'}), 
                  how='left', on=['grant_id_2', 'year'])
    grnts_info.columns = [col[:-2] for col in list(grnts_info)]

    cols = ['grant_id', 'rsr_id', 'status']
    if yearly == True:
        cols += ['year']
    grnts_stats = df.groupby(cols)
    grnts_stats = pd.DataFrame({'nb_grnts':grnts_stats['grant_id_2'].nunique()
                                , 'fund_amt':grnts_stats['funding_amount_2'].mean()
                                , 'avg_fund_len':grnts_stats['funding_len_2'].mean()
                                , 'avg_team_size':grnts_stats['nb_rsrs_2'].mean()
                               }).reset_index()
    grnts_stats = pd.merge(funded_grnts_funder, grnts_stats, how='left', on='grant_id')
    
    return grnts_stats

In [104]:
def grant_statistics(grnts_stats):
    
    print("Pre-Funding Grant Statistics:")
    df = grnts_stats[grnts_stats['status']=="pre"].copy()
    grnts_pre_avg_len_avg = []
    grnts_pre_avg_len_med = []
    grnts_pre_avg_team_size_avg = []
    grnts_pre_avg_team_size_med = []
    grnts_pre_avg_amt_avg = []
    grnts_pre_avg_amt_med = []
    grnts_pre_nb_grnts_avg = []
    grnts_pre_nb_grnts_med = []
    cols = ['nb_grnts', 'fund_amt', 'avg_fund_len', 'avg_team_size']
    for funder in funders:
        print("\n{}:".format(funder))
        temp = df[df['funder_name']==funder]
        print(temp[cols].describe())
        grnts_pre_avg_len_avg.append(temp['avg_fund_len'].mean())
        grnts_pre_avg_len_med.append(temp['avg_fund_len'].median())
        grnts_pre_avg_team_size_avg.append(temp['avg_team_size'].mean())
        grnts_pre_avg_team_size_med.append(temp['avg_team_size'].median())
        grnts_pre_avg_amt_avg.append(temp['fund_amt'].mean())
        grnts_pre_avg_amt_med.append(temp['fund_amt'].median())
        grnts_pre_nb_grnts_avg.append(temp['nb_grnts'].mean())
        grnts_pre_nb_grnts_med.append(temp['nb_grnts'].median())
        
    print("\n")
    
    print("Post-Funding Grant Statistics:")
    df = grnts_stats[grnts_stats['status']=="post"].copy()
    grnts_post_avg_len_avg = []
    grnts_post_avg_len_med = []
    grnts_post_avg_team_size_avg = []
    grnts_post_avg_team_size_med = []
    grnts_post_avg_amt_avg = []
    grnts_post_avg_amt_med = []
    grnts_post_nb_grnts_avg = []
    grnts_post_nb_grnts_med = []
    cols = ['nb_grnts', 'fund_amt', 'avg_fund_len', 'avg_team_size']
    for funder in funders:
        print("\n{}:".format(funder))
        temp = df[df['funder_name']==funder]
        print(temp[cols].describe())
        grnts_post_avg_len_avg.append(temp['avg_fund_len'].mean())
        grnts_post_avg_len_med.append(temp['avg_fund_len'].median())
        grnts_post_avg_team_size_avg.append(temp['avg_team_size'].mean())
        grnts_post_avg_team_size_med.append(temp['avg_team_size'].median())
        grnts_post_avg_amt_avg.append(temp['fund_amt'].mean())
        grnts_post_avg_amt_med.append(temp['fund_amt'].median())
        grnts_post_nb_grnts_avg.append(temp['nb_grnts'].mean())
        grnts_post_nb_grnts_med.append(temp['nb_grnts'].median())

    return [grnts_pre_avg_len_avg, grnts_pre_avg_len_med, grnts_pre_avg_team_size_avg, grnts_pre_avg_team_size_med, 
            grnts_pre_avg_amt_avg, grnts_pre_avg_amt_med, grnts_pre_nb_grnts_avg, grnts_pre_nb_grnts_med, 
            grnts_post_avg_len_avg, grnts_post_avg_len_med, grnts_post_avg_team_size_avg, 
            grnts_post_avg_team_size_med, grnts_post_avg_amt_avg, grnts_post_avg_amt_med, grnts_post_nb_grnts_avg, 
            grnts_post_nb_grnts_med]

In [105]:
grnts_stats = grant_groupby(grnts_mrg)

In [106]:
temp = grnts_stats[grnts_stats['status']=="pre"].copy()
print("Average Funding Length Pre-Grant: {}".format(temp['avg_fund_len'].mean()))
grnts_avg_len_tot = [temp['avg_fund_len'].mean(), temp['avg_fund_len'].std()]
print("Average Team Size Pre-Grant: {}".format(temp['avg_team_size'].mean()))
grnts_avg_team_size_tot = [temp['avg_team_size'].mean(), temp['avg_team_size'].std()]
print("Average Funding Amount Pre-Grant: {}".format(temp['fund_amt'].mean()))
grnts_avg_amt_tot = [temp['fund_amt'].mean(), temp['fund_amt'].std()]
print("Average Total Number of Grants Pre-Grant: {}".format(temp['nb_grnts'].mean()))
grnts_nb_grnts_tot = [temp['nb_grnts'].mean(), temp['nb_grnts'].std()]

temp = grnts_stats[grnts_stats['status']=="post"].copy()
print("\nAverage Funding Length Post-Grant: {}".format(temp['avg_fund_len'].mean()))
grnts_avg_len_tot = [temp['avg_fund_len'].mean(), temp['avg_fund_len'].std()]
print("Average Team Size Post-Grant: {}".format(temp['avg_team_size'].mean()))
grnts_avg_team_size_tot = [temp['avg_team_size'].mean(), temp['avg_team_size'].std()]
print("Average Funding Amount Post-Grant: {}".format(temp['fund_amt'].mean()))
grnts_avg_amt_tot = [temp['fund_amt'].mean(), temp['fund_amt'].std()]
print("Average Total Number of Grants Post-Grant: {}".format(temp['nb_grnts'].mean()))
grnts_nb_grnts_tot = [temp['nb_grnts'].mean(), temp['nb_grnts'].std()]

print("\n")

grnt_stats = grant_statistics(grnts_stats)

# avg_fund_len_avg = output[0]
# avg_fund_len_med = output[1]
# avg_team_size_avg = output[2]
# avg_team_size_med = output[3]
# fund_amt_avg = output[4]
# fund_amt_med = output[5]
# nb_grnts_avg = output[6]
# nb_grnts_med = output[7]
# avg_fund_len_avg = output[8]
# avg_fund_len_med = output[9]
# avg_team_size_avg = output[10]
# avg_team_size_med = output[11]
# fund_amt_avg = output[12]
# fund_amt_med = output[13]
# nb_grnts_avg = output[14]
# nb_grnts_med = output[15]

Average Funding Length Pre-Grant: 3.9050025362265837
Average Team Size Pre-Grant: 2.4040574645643296
Average Funding Amount Pre-Grant: 1989366.909688983
Average Total Number of Grants Pre-Grant: 1.6462080809906896

Average Funding Length Post-Grant: 3.67896111845694
Average Team Size Post-Grant: 2.604911313285655
Average Funding Amount Post-Grant: 1903026.0359355172
Average Total Number of Grants Post-Grant: 1.7843713278495887


Pre-Funding Grant Statistics:

Cancer Research UK:
          nb_grnts      fund_amt  avg_fund_len  avg_team_size
count  2291.000000  7.680000e+02   1368.000000    1369.000000
mean      2.243998  1.452412e+06      3.784609       1.943205
std       2.978130  2.095033e+06      1.440834       1.912171
min       0.000000  2.265500e+03      0.082192       1.000000
25%       0.000000  3.574960e+05      3.000000       1.000000
50%       1.000000  7.293998e+05      3.605577       1.000000
75%       3.000000  1.763075e+06      4.452055       2.111111
max      19.000000  

## 3. Prior and Subsequent Publications

In [107]:
print("Average Number of Citations: {}".format(pubs_info['citations'].mean()))
pubs_cit_tot = [pubs_info['citations'].mean(), pubs_info['citations'].std()]
print("Average Publication Team Size: {}".format(pubs_info['nb_authors'].mean()))
pubs_team_size_tot = [pubs_info['nb_authors'].mean(), pubs_info['nb_authors'].std()]

Average Number of Citations: 31.136625850340135
Average Publication Team Size: 9.038140589569162


In [108]:
def pub_groupby(pubs_mrg, yearly=False):
    
    cols = ['grant_id', 'rsr_id', 'status']
    if yearly == True:
        cols += ['year']
    
    # Publication Statistics
    df = pd.merge(pubs_mrg, pubs_info, how='left', on=['pub_id', 'year'])
    pubs_stats = df.groupby(cols)
    pubs_stats = pd.DataFrame({'nb_pubs':pubs_stats['pub_id'].nunique()
                                , 'citations_per_pub':pubs_stats['citations'].mean()
                                , 'team_size':pubs_stats['nb_authors'].mean()
                               }).reset_index()
    
    # Network Statistics:
    df = pd.merge(pubs_mrg, pubs_rsr.rename(columns={'rsr_id':'author_id'}), 
                  how='left', on='pub_id')
    pubs_collab = df.groupby(cols)
    pubs_collab = pd.DataFrame({'nb_collabs': pubs_collab['author_id'].nunique()
                                , 'nb_collab_countries': pubs_collab['rsr_country'].nunique()
                               }).reset_index()
    
    # Combine the two:
    pubs_stats = pd.merge(pubs_stats, pubs_collab, how='outer', on=cols)
    pubs_stats = pd.merge(funded_grnts_funder, pubs_stats, how='left', on='grant_id')
    
    return pubs_stats

In [109]:
def publication_statistics(pubs_stats):
    
    print("Pre-Funding Publication Statistics:")
    df = pubs_stats[pubs_stats['status']=="pre"].copy()
    pubs_pre_cit_avg = []
    pubs_pre_cit_med = []
    pubs_pre_nb_pubs_avg = []
    pubs_pre_nb_pubs_med = []
    pubs_pre_team_size_avg = []
    pubs_pre_team_size_med = []
    cols = ['citations_per_pub', 'nb_pubs', 'team_size', 'nb_collabs', 'nb_collab_countries']
    for funder in funders:
        print("\n{}:".format(funder))
        temp = pubs_stats[pubs_stats['funder_name']==funder]
        print(temp[cols].describe())
        pubs_pre_cit_avg.append(temp['citations_per_pub'].mean())
        pubs_pre_cit_med.append(temp['citations_per_pub'].median())
        pubs_pre_nb_pubs_avg.append(temp['nb_pubs'].mean())
        pubs_pre_nb_pubs_med.append(temp['nb_pubs'].median())
        pubs_pre_team_size_avg.append(temp['team_size'].mean())
        pubs_pre_team_size_med.append(temp['team_size'].median())
    
    print("\n")
    
    print("Post-Funding Publication Statistics:")
    df = pubs_stats[pubs_stats['status']=="post"].copy()
    pubs_post_cit_avg = []
    pubs_post_cit_med = []
    pubs_post_nb_pubs_avg = []
    pubs_post_nb_pubs_med = []
    pubs_post_team_size_avg = []
    pubs_post_team_size_med = []
    cols = ['citations_per_pub', 'nb_pubs', 'team_size', 'nb_collabs', 'nb_collab_countries']
    for funder in funders:
        print("\n{}:".format(funder))
        temp = pubs_stats[pubs_stats['funder_name']==funder]
        print(temp[cols].describe())
        pubs_pre_cit_avg.append(temp['citations_per_pub'].mean())
        pubs_pre_cit_med.append(temp['citations_per_pub'].median())
        pubs_pre_nb_pubs_avg.append(temp['nb_pubs'].mean())
        pubs_pre_nb_pubs_med.append(temp['nb_pubs'].median())
        pubs_pre_team_size_avg.append(temp['team_size'].mean())
        pubs_pre_team_size_med.append(temp['team_size'].median())
        
    return [pubs_pre_cit_avg, pubs_pre_cit_med, pubs_pre_nb_pubs_avg, pubs_pre_nb_pubs_med, pubs_pre_team_size_avg, 
            pubs_pre_team_size_med, pubs_post_cit_avg, pubs_post_cit_med, pubs_post_nb_pubs_avg, pubs_post_nb_pubs_med,
            pubs_post_team_size_avg, pubs_post_team_size_med]

In [110]:
pubs_stats = pub_groupby(pubs_mrg)

In [111]:
temp = pubs_stats[pubs_stats['status']=="pre"].copy()
print("Average Citations Pre-Grant: {}".format(temp['citations_per_pub'].mean()))
pub_cit_tot = [temp['citations_per_pub'].mean(), temp['citations_per_pub'].std()]
print("Average Total Number of Publications Pre-Grant: {}".format(temp['nb_pubs'].mean()))
nb_pubs_tot = [temp['nb_pubs'].mean(), temp['nb_pubs'].std()]

temp = pubs_stats[pubs_stats['status']=="post"].copy()
print("\nAverage Citations Post-Grant: {}".format(temp['citations_per_pub'].mean()))
pub_cit_tot = [temp['citations_per_pub'].mean(), temp['citations_per_pub'].std()]
print("Average Total Number of Publications Post-Grant: {}".format(temp['nb_pubs'].mean()))
nb_pubs_tot = [temp['nb_pubs'].mean(), temp['nb_pubs'].std()]

print("\n")

pub_stats = publication_statistics(pubs_stats)

# citations_avg = output[0]
# nb_pubs_avg = output[1]
# citations_med = output[2]
# nb_pubs_med = output[3]
# citations_avg = output[4]
# citations_med = output[5]
# nb_pubs_avg = output[6]
# nb_pubs_med = output[7]
# team_size_avg = output[8]
# team_size_med = output[9]
# team_size_avg = output[10]
# team_size_med = output[11]

Average Citations Pre-Grant: 99.88162101576859
Average Total Number of Publications Pre-Grant: 2.7823375214679564

Average Citations Post-Grant: 108.20780121797493
Average Total Number of Publications Post-Grant: 4.506643767513332


Pre-Funding Publication Statistics:

Cancer Research UK:
       citations_per_pub      nb_pubs    team_size   nb_collabs  \
count        1149.000000  6873.000000  1149.000000  6873.000000   
mean          147.829759     0.569620    69.580453    14.385276   
std           328.557644     2.667414   278.655156    98.484123   
min             0.000000     0.000000     2.000000     0.000000   
25%            25.500000     0.000000    10.000000     0.000000   
50%            53.608696     0.000000    15.000000     0.000000   
75%           118.500000     0.000000    20.500000     0.000000   
max          2537.000000    75.000000  2467.000000  2024.000000   

       nb_collab_countries  
count          6873.000000  
mean              1.075804  
std               4

       citations_per_pub      nb_pubs    team_size   nb_collabs  \
count         992.000000  9336.000000   992.000000  9336.000000   
mean          136.467441     0.391388    73.977257    13.915060   
std           314.169987     2.037092   211.124560    95.098581   
min             0.000000     0.000000     2.000000     0.000000   
25%            18.125000     0.000000    12.000000     0.000000   
50%            52.000000     0.000000    17.833333     0.000000   
75%           117.375000     0.000000    53.750000     0.000000   
max          3583.000000    44.000000  2467.000000  1719.000000   

       nb_collab_countries  
count          9336.000000  
mean              1.017459  
std               4.496981  
min               0.000000  
25%               0.000000  
50%               0.000000  
75%               0.000000  
max              52.000000  

Wellcome Trust:
       citations_per_pub      nb_pubs   team_size   nb_collabs  \
count         160.000000  1248.000000  160.000000  1

## 4. Subsequent Publication RCDC Codes

### Create Lookup Tables

In [112]:
# grnts_rcdc_1 = grnts_rcdc.drop_duplicates('grant_id', keep='first').reset_index(drop=True)
# pubs_rcdc_1 = pubs_rcdc.drop_duplicates('pub_id', keep='first').reset_index(drop=True)

### Method 1: 1-to-1

In [113]:
# cols = ['funder_name', 'funding_amount', 'grant_id', 'rsr_id', 'start_date', 'pub_id', 'citations']
# rcdc_comp = pubs_mrg[cols].copy()

In [114]:
# rcdc_comp = pd.merge(rcdc_comp, funded_grnt_rcdc_1, how='left', on='grant_id', suffixes=('', '_1_grnt'))
# rcdc_comp = pd.merge(rcdc_comp, pub_rcdc_1, how='left', on='pub_id', suffixes=('', '_1_pub'))
# rcdc_comp.rename(columns={'rcdc_name': 'rcdc_name_1_grnt', 'rcdc_code': 'rcdc_code_1_grnt'}, inplace=True)

In [115]:
# # Get rid or ignore Null Values?
# process = 'ignore' # or 'get_rid'
# if process == 'get_rid':
#     rcdc_comp = rcdc_1_comp[(rcdc_comp['rcdc_name_1_grnt'].notnull())&(rcdc_comp['rcdc_name_1_pub'].notnull())]
# elif process == 'ignore':
#     rcdc_comp['rcdc_name_1_grnt'] = rcdc_comp['rcdc_name_1_grnt'].fillna('')
#     rcdc_comp['rcdc_name_1_pub'] = rcdc_comp['rcdc_name_1_pub'].fillna('')
# else:
#     print("Don't forget to choose !")

In [116]:
# rcdc_comp_agency = rcdc_comp.groupby(['funder_name', 'rcdc_name_1_grnt', 'rcdc_name_1_pub'])
# rcdc_comp_agency = pd.DataFrame({'nb_obs': rcdc_comp_agency.size()
#                                    , 'nb_grnts': rcdc_comp_agency['grant_id'].nunique()
#                                    , 'nb_pubs': rcdc_comp_agency['pub_id'].nunique()
#                                   }).reset_index()

In [117]:
# rcdc_comp_agency.head()

In [118]:
# rcdc_comp_agency.tail()

In [119]:
# rcdc_comp_agency.columns = [['funder_name', 'grnt_rcdc', 'pub_rcdc', 'nb_grnts', 'nb_obs', 'nb_pubs']]
# rcdc_comp_agency.to_csv('../output/rcdc_grnt_1_pubs_1_comp.csv', index=False)

### Method 2: 1 to Many

In [120]:
# cols = ['funder_name', 'funding_amount', 'grant_id', 'rsr_id', 'start_date', 'pub_id', 'citations']
# cols += ['rcdc_names_pub', 'rcdc_codes_pub']
# rcdc_comp = pubs_mrg[cols].copy()

In [121]:
# rcdc_comp = pd.merge(rcdc_comp, funded_grnt_rcdc_1, how='left', on='grant_id', suffixes=('', '_1_grnt'))
# rcdc_comp = pd.merge(rcdc_comp, pub_rcdc_1, how='left', on='pub_id', suffixes=('', '_1_pub'))
# rcdc_comp.rename(columns={'rcdc_name': 'rcdc_name_1_grnt', 'rcdc_code': 'rcdc_code_1_grnt'}, inplace=True)

In [122]:
# # Get rid or ignore Null Values?
# process = 'ignore' # or 'get_rid'
# if process == 'get_rid':
#     rcdc_comp = rcdc_1_comp[(rcdc_comp['rcdc_name_1_grnt'].notnull())&(rcdc_comp['rcdc_names_pub'].notnull())]
# elif process == 'ignore':
#     rcdc_comp['rcdc_name_1_grnt'] = rcdc_comp['rcdc_name_1_grnt'].fillna('')
#     rcdc_comp['rcdc_names_pub'] = rcdc_comp['rcdc_names_pub'].fillna('')
#     rcdc_comp['rcdc_name_1_pub'] = rcdc_comp['rcdc_name_1_pub'].fillna('')
# else:
#     print("Don't forget to choose !")

In [123]:
# def regin(df):
#     return bool(re.search(r"(^|; )\b{}\b(; |$)".format(df['rcdc_name_1_grnt']), df['rcdc_names_pub']))
# rcdc_comp['flag'] = rcdc_comp.apply(regin, axis=1)

In [124]:
# rcdc_comp['rcdc_name_pub_impute'] = np.where(rcdc_comp['flag']==True
#                                              , rcdc_comp['rcdc_name_1_grnt'], rcdc_comp['rcdc_name_1_pub'])

In [125]:
# rcdc_comp_agency = rcdc_comp.groupby(['funder_name', 'rcdc_name_1_grnt', 'rcdc_name_pub_impute'])
# rcdc_comp_agency = pd.DataFrame({'nb_obs': rcdc_comp_agency.size()
#                                  , 'nb_grnts': rcdc_comp_agency['grant_id'].nunique()
#                                  , 'nb_pubs': rcdc_comp_agency['pub_id'].nunique()
#                                 }).reset_index()

In [126]:
# rcdc_comp_agency.head()

In [127]:
# rcdc_comp_agency.tail()

In [128]:
# rcdc_comp_agency.columns = [['funder_name', 'grnt_rcdc', 'pub_rcdc', 'nb_grnts', 'nb_obs', 'nb_pubs']]
# rcdc_comp_agency.to_csv('../output/rcdc_grnt_1_pubs_comp.csv', index=False)

### Method 3: Many to Many

In [129]:
# cols = ['funder_name', 'funding_amount', 'grant_id', 'rsr_id', 'start_date', 'pub_id', 'citations']
# cols += ['rcdc_names_pub', 'rcdc_codes_pub']
# rcdc_comp = pubs_mrg[cols].copy()

In [130]:
# rcdc_comp = pd.merge(rcdc_comp, funded_grnt_rcdc, how='left', on='grant_id', suffixes=('', '_grnt'))
# rcdc_comp = pd.merge(rcdc_comp, pub_rcdc_1, how='left', on='pub_id', suffixes=('', '_1_pub'))
# rcdc_comp.rename(columns={'rcdc_name': 'rcdc_name_grnt', 'rcdc_code': 'rcdc_code_grnt'}, inplace=True)

In [131]:
# # Get rid or ignore Null Values?
# process = 'ignore' # or 'get_rid'
# if process == 'get_rid':
#     rcdc_comp = rcdc_1_comp[(rcdc_comp['rcdc_name_grnt'].notnull())&(rcdc_comp['rcdc_names_pub'].notnull())]
# elif process == 'ignore':
#     rcdc_comp['rcdc_name_grnt'] = rcdc_comp['rcdc_name_grnt'].fillna('')
#     rcdc_comp['rcdc_names_pub'] = rcdc_comp['rcdc_names_pub'].fillna('')
#     rcdc_comp['rcdc_name_1_pub'] = rcdc_comp['rcdc_name_1_pub'].fillna('')
# else:
#     print("Don't forget to choose !")

In [132]:
# def regin(df):
#     return bool(re.search(r"(^|; )\b{}\b(; |$)".format(df['rcdc_name_grnt']), df['rcdc_names_pub']))
# rcdc_comp['flag'] = rcdc_comp.apply(regin, axis=1)

In [133]:
# rcdc_comp['rcdc_name_pub_impute'] = np.where(rcdc_comp['flag']==True
#                                              , rcdc_comp['rcdc_name_grnt'], rcdc_comp['rcdc_name_1_pub'])

In [134]:
# rcdc_comp_agency = rcdc_comp.groupby(['funder_name', 'rcdc_name_grnt', 'rcdc_name_pub_impute'])
# rcdc_comp_agency = pd.DataFrame({'nb_obs': rcdc_comp_agency.size()
#                                  , 'nb_grnts': rcdc_comp_agency['grant_id'].nunique()
#                                  , 'nb_pubs': rcdc_comp_agency['pub_id'].nunique()
#                                 }).reset_index()

In [135]:
# rcdc_comp_agency.head()

In [136]:
# rcdc_comp_agency.tail()

In [137]:
# rcdc_comp_agency.columns = [['funder_name', 'grnt_rcdc', 'pub_rcdc', 'nb_grnts', 'nb_obs', 'nb_pubs']]
# rcdc_comp_agency.to_csv('../output/rcdc_grnt_pubs_comp.csv', index=False)

## 5. Publication CSO Codes

In [138]:
pubs_cso_1 = pubs_cso.drop_duplicates('pub_id', keep='first').reset_index(drop=True)

### Most Common CSO Codes by Funding Agency

In [139]:
df = pd.merge(pubs_mrg, pubs_cso, how='left', on='pub_id')
df = df[['funder_name', 'pub_id', 'cso', 'nb_cso']].drop_duplicates().reset_index(drop=True)

In [140]:
print("Average number of CSO Codes per Publication Associated to Grant:\n")
print(df[['funder_name', 'pub_id', 'nb_cso']].drop_duplicates().groupby('funder_name')['nb_cso'].mean())

Average number of CSO Codes per Publication Associated to Grant:

funder_name
Cancer Research UK                              1.420561
INCa/INSERM/DGOS                                1.324357
INCa/INSERM/DGOS - ORCID Confirmed              1.308430
National Cancer Institute                       1.414898
National Health and Medical Research Council    1.375817
Wellcome Trust                                  1.598765
Name: nb_cso, dtype: float64


In [141]:
cso_1 = []
cso_2 = []
cso_3 = []
cso_4 = []
cso_5 = []

for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder]
    print("Number of Publications with missing CSO Codes: {}".format(temp[temp['cso'].isnull()].shape[0]))
    print(temp['cso'].value_counts().head())
    cso_1.append(temp['cso'].value_counts().index[0])
    cso_2.append(temp['cso'].value_counts().index[1])
    cso_3.append(temp['cso'].value_counts().index[2])
    cso_4.append(temp['cso'].value_counts().index[3])
    cso_5.append(temp['cso'].value_counts().index[4])


Cancer Research UK:
Number of Publications with missing CSO Codes: 545
5.4    244
2.2     88
5.3     88
2.1     79
2.3     72
Name: cso, dtype: int64

INCa/INSERM/DGOS:
Number of Publications with missing CSO Codes: 31253
5.4    3516
1.1    2930
5.3    2926
2.1    2457
4.3    1896
Name: cso, dtype: int64

INCa/INSERM/DGOS - ORCID Confirmed:
Number of Publications with missing CSO Codes: 6617
1.1    1213
5.3     973
5.4     948
2.1     569
1.4     508
Name: cso, dtype: int64

National Cancer Institute:
Number of Publications with missing CSO Codes: 2969
2.1    593
5.4    466
2.3    260
2.2    243
5.3    227
Name: cso, dtype: int64

National Health and Medical Research Council:
Number of Publications with missing CSO Codes: 397
5.4    155
2.2    113
2.1     95
2.3     84
5.3     68
Name: cso, dtype: int64

Wellcome Trust:
Number of Publications with missing CSO Codes: 100
2.2    63
2.3    40
5.3    24
1.3    19
5.4    18
Name: cso, dtype: int64


## Creating Output Comparison Table

### Overall Table

In [ ]:
df = pd.DataFrame({
    'funded_grnts_per_rsrs_tot': funded_grnts_per_rsrs_tot
    , 'agencies_per_rsrs_tot': agencies_per_rsrs_tot
    , 'funded_amt_tot': funded_amt_tot
    , 'funded_len_tot': funded_len_tot
    , 'nb_grnt_rsrs_tot': nb_grnt_rsrs_tot
    , 'grnt_fund_len_tot': grnt_fund_len_tot
    , 'grnt_team_size_tot': grnt_team_size_tot
    , 'grnt_fund_amt_tot': grnt_fund_amt_tot
    , 'pre_avg_fund_len_tot': pre_avg_fund_len_tot
    , 'pre_avg_team_size_tot': pre_avg_team_size_tot
    , 'pre_fund_amt_tot': pre_fund_amt_tot
    , 'pre_nb_grnts_tot': pre_nb_grnts_tot
    , 'post_avg_fund_len_tot': post_avg_fund_len_tot
    , 'post_avg_team_size_tot': post_avg_team_size_tot
    , 'post_fund_amt_tot': post_fund_amt_tot
    , 'post_nb_grnts_tot': post_nb_grnts_tot
    , 'pub_cit_tot': pub_cit_tot
    , 'pub_team_size_tot': pub_team_size_tot
    , 'pre_pub_cit_tot': pre_pub_cit_tot
    , 'pre_nb_pubs_tot': pre_nb_pubs_tot
    , 'post_pub_cit_tot': post_pub_cit_tot
    , 'post_nb_pubs_tot': post_nb_pubs_tot
    }, index=['mean', 'std']).transpose()
df

In [ ]:
# Export to Excel
ls = !ls ../output/
if 'comparison_statistics.xlsx' in ls:
    book = load_workbook('../output/comparison_statistics.xlsx')
    writer = pd.ExcelWriter('../output/comparison_statistics.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, "raw_all")
    writer.save()
else:
    df.to_excel('../output/comparison_statistics.xlsx', sheet_name = 'raw_all')

### Table by Funding Agency

Table with funders as columns and:
- 1st RCDC Code
- 2nd RCDC Code
- 3rd RCDC Code
- Mean number of previous grants
- Median number of previous grants
- Mean amount of previous grants
- Median number of previous grants
- Mean length of previous grants
- Median length of previous grants

In [ ]:
df = pd.DataFrame({'rcdc_1':rcdc_1
                    , 'rcdc_2':rcdc_2
                    , 'rcdc_3':rcdc_3
                    , 'rcdc_4':rcdc_4
                    , 'rcdc_5':rcdc_5                   
                    , 'nb_unique_rsrs': nb_unique_rsrs
                    , 'nb_unique_grnts': nb_unique_grnts
                    , 'funded_amt_avg': funded_amt_avg
                    , 'funded_amt_med': funded_amt_med
                    , 'funded_len_avg': funded_len_avg
                    , 'funded_len_med': funded_len_med
                    , 'nb_grnt_rsrs_avg': nb_grnt_rsrs_avg
                    , 'nb_grnt_rsrs_med': nb_grnt_rsrs_med
                    , 'pre_avg_fund_len_avg': pre_avg_fund_len_avg
                    , 'pre_avg_fund_len_med': pre_avg_fund_len_med
                    , 'pre_avg_team_size_avg': pre_avg_team_size_avg
                    , 'pre_avg_team_size_med': pre_avg_team_size_med
                    , 'pre_fund_amt_avg': pre_fund_amt_avg
                    , 'pre_fund_amt_med': pre_fund_amt_med
                    , 'pre_nb_grnts_avg': pre_nb_grnts_avg
                    , 'pre_nb_grnts_med': pre_nb_grnts_med
                    , 'post_avg_fund_len_avg': post_avg_fund_len_avg
                    , 'post_avg_fund_len_med': post_avg_fund_len_med
                    , 'post_avg_team_size_avg': post_avg_team_size_avg
                    , 'post_avg_team_size_med': post_avg_team_size_med
                    , 'post_fund_amt_avg': post_fund_amt_avg
                    , 'post_fund_amt_med': post_fund_amt_med
                    , 'post_nb_grnts_avg': post_nb_grnts_avg
                    , 'post_nb_grnts_med': post_nb_grnts_med
                    , 'pre_citations_avg': pre_citations_avg
                    , 'pre_nb_pubs_avg': pre_nb_pubs_avg
                    , 'pre_citations_med': pre_citations_med
                    , 'pre_nb_pubs_med': pre_nb_pubs_med
                    , 'post_citations_avg': post_citations_avg
                    , 'post_citations_med': post_citations_med
                    , 'post_nb_pubs_avg': post_nb_pubs_avg
                    , 'post_nb_pubs_med': post_nb_pubs_med
                   , 'pre_team_size_avg': pre_team_size_avg
                   , 'pre_team_size_med': pre_team_size_med
                   , 'post_team_size_avg': post_team_size_avg
                   , 'post_team_size_med': post_team_size_med
                  }, index=funders).transpose()
df

In [ ]:
# Export to Excel
ls = !ls ../output/
if 'comparison_statistics.xlsx' in ls:
    book = load_workbook('../output/comparison_statistics.xlsx')
    writer = pd.ExcelWriter('../output/comparison_statistics.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, "raw_by_agency")
    writer.save()
else:
    df.to_excel('../output/comparison_statistics.xlsx', sheet_name = 'raw_by_agency')

## Preparing Data for Regression Analysis

In [142]:
yearly = True

### Grant Information

In [143]:
print(funded_grnts_info.set_index(['grant_id']).index.is_unique)
print(funded_grnts_info.shape)
funded_grnts_info.head()

True
(16926, 7)


,end_date,funding_amount,grant_id,start_date,funding_len,nb_rsrs,start_year
0,2018-11-30,3181288.0,grant.2482176,2012-12-31,5.917808,3,2012
1,2017-11-30,896400.0,grant.2411281,2012-12-18,4.953425,1,2012
2,2013-12-16,45010.0,grant.2344785,2012-12-17,0.997260,1,2012
3,2018-01-31,1264455.0,grant.2482260,2012-12-15,5.131507,1,2012
4,2015-03-30,157600.0,grant.2569907,2012-12-15,2.287671,1,2012


### Prior/Subsequent Grant Statistics

In [144]:
grnts_stats = grant_groupby(grnts_mrg, yearly=yearly)

In [145]:
# Check if unit of observation is Funder-Grant-Researcher:
cols = ['funder_name', 'grant_id', 'rsr_id', 'status']
if yearly == True:
    cols+=['year']
print(grnts_stats.set_index(cols).index.is_unique)
print(grnts_stats.shape)
grnts_stats.head()

True
(243386, 9)


,funder_name,grant_id,rsr_id,status,year,avg_fund_len,avg_team_size,fund_amt,nb_grnts
0,National Cancer Institute,grant.2482176,ur.015107420202.13,focal,2012,5.917808,3.0,3181288.0,1
1,National Cancer Institute,grant.2482176,ur.015107420202.13,post,2013,NaN,NaN,NaN,0
2,National Cancer Institute,grant.2482176,ur.015107420202.13,post,2014,NaN,NaN,NaN,0
3,National Cancer Institute,grant.2482176,ur.015107420202.13,post,2015,NaN,NaN,NaN,0
4,National Cancer Institute,grant.2482176,ur.015107420202.13,post,2016,5.000000,2.0,1107840.0,1


### Prior/Subsequent Publication Statistics

In [146]:
pubs_stats = pub_groupby(pubs_mrg, yearly=yearly)

In [147]:
# Check if unit of observation is Funder-Grant-Researcher:
cols = ['funder_name', 'grant_id', 'rsr_id', 'status']
if yearly == True:
    cols+=['year']
print(pubs_stats.set_index(cols).index.is_unique)
print(pubs_stats.shape)
pubs_stats.head()

True
(243386, 10)


,funder_name,grant_id,rsr_id,status,year,citations_per_pub,nb_pubs,team_size,nb_collab_countries,nb_collabs
0,National Cancer Institute,grant.2482176,ur.015107420202.13,focal,2012,4.400000,5,5.400000,2,18
1,National Cancer Institute,grant.2482176,ur.015107420202.13,post,2013,4.400000,10,4.500000,3,25
2,National Cancer Institute,grant.2482176,ur.015107420202.13,post,2014,6.857143,7,4.285714,2,16
3,National Cancer Institute,grant.2482176,ur.015107420202.13,post,2015,5.000000,3,3.666667,2,7
4,National Cancer Institute,grant.2482176,ur.015107420202.13,post,2016,5.625000,8,6.625000,3,28


### Add CSO Codes

In [148]:
pubs_cso_features = pd.merge(pubs_mrg[['funder_name', 'grant_id', 'rsr_id', 'pub_id', 'status']], pubs_cso, 
                             how='inner', on='pub_id')
pubs_cso_features['cso'] = "cso_"+pubs_cso_features['cso'].astype(str)

In [149]:
pubs_cso_features = pubs_cso_features.groupby(['funder_name', 'grant_id', 'rsr_id', 'status', 'cso'])
pubs_cso_features = pd.DataFrame({'nb_pubs': pubs_cso_features['pub_id'].count()}).reset_index()

In [150]:
pubs_cso_features = pd.pivot_table(pubs_cso_features, index=['funder_name', 'grant_id', 'rsr_id'], 
                                   columns=['status', 'cso'], values='nb_pubs', aggfunc='count').reset_index()
pubs_cso_features.columns = ['_'.join(col).rstrip('_') for col in pubs_cso_features.columns.values]
pubs_cso_features.fillna(0, inplace=True)

In [151]:
print(pubs_cso_features.set_index(['funder_name', 'grant_id', 'rsr_id']).index.is_unique)
print(pubs_cso_features.shape)
pubs_cso_features.head()

True
(4540, 101)


,funder_name,grant_id,rsr_id,focal_cso_1.1,focal_cso_1.2,focal_cso_1.3,focal_cso_1.4,focal_cso_1.5,focal_cso_2.1,focal_cso_2.2,focal_cso_2.3,focal_cso_2.4,focal_cso_3.1,focal_cso_3.2,focal_cso_3.3,focal_cso_3.4,focal_cso_3.6,focal_cso_4.1,focal_cso_4.2,focal_cso_4.3,focal_cso_4.4,focal_cso_5.1,focal_cso_5.2,focal_cso_5.3,focal_cso_5.4,focal_cso_5.5,focal_cso_5.7,focal_cso_6.1,focal_cso_6.2,focal_cso_6.3,focal_cso_6.4,focal_cso_6.5,focal_cso_6.6,focal_cso_6.7,focal_cso_6.9,post_cso_1.1,post_cso_1.2,post_cso_1.3,post_cso_1.4,post_cso_1.5,post_cso_2.1,post_cso_2.2,post_cso_2.3,post_cso_2.4,post_cso_3.1,post_cso_3.2,post_cso_3.3,post_cso_3.4,post_cso_3.5,post_cso_3.6,...,post_cso_4.2,post_cso_4.3,post_cso_4.4,post_cso_5.1,post_cso_5.2,post_cso_5.3,post_cso_5.4,post_cso_5.5,post_cso_5.6,post_cso_5.7,post_cso_6.1,post_cso_6.2,post_cso_6.3,post_cso_6.4,post_cso_6.5,post_cso_6.6,post_cso_6.7,post_cso_6.9,pre_cso_1.1,pre_cso_1.2,pre_cso_1.3,pre_cso_1.4,pre_cso_1.5,pre_cso_2.1,pre_cso_2.2,pre_cso_2.3,pre_cso_2.4,pre_cso_3.1,pre_cso_3.2,pre_cso_3.3,pre_cso_3.4,pre_cso_4.1,pre_cso_4.2,pre_cso_4.3,pre_cso_4.4,pre_cso_5.1,pre_cso_5.2,pre_cso_5.3,pre_cso_5.4,pre_cso_5.5,pre_cso_5.6,pre_cso_5.7,pre_cso_6.1,pre_cso_6.2,pre_cso_6.3,pre_cso_6.4,pre_cso_6.5,pre_cso_6.6,pre_cso_6.7,pre_cso_6.9
0,Cancer Research UK,grant.5133993,ur.01342746606.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Cancer Research UK,grant.5133999,ur.015105521212.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Cancer Research UK,grant.5134023,ur.01155754475.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cancer Research UK,grant.5134045,ur.01360534371.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cancer Research UK,grant.5134047,ur.01132707630.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Combine All

In [152]:
df = pd.merge(funded_grnts[['funder_name', 'grant_id', 'start_year', 'start_date', 'rsr_id']].drop_duplicates(), 
              merge_key, how='left', on='start_year')
df = pd.merge(funded_grnts_funder, funded_grnts_rsr, how='outer', on='grant_id')
df = pd.merge(df, funded_grnts_info, how='outer', on='grant_id')
df = pd.merge(df, merge_key, how='left', on='start_year')
df = pd.merge(df, grnts_stats, how='outer', on=['funder_name', 'grant_id', 'rsr_id', 'year', 'status'])
df = pd.merge(df, pubs_stats, how='outer', on=['funder_name', 'grant_id', 'rsr_id', 'year', 'status'])
# df = pd.merge(df, pubs_cso_features, how='outer', on=['funder_name', 'grant_id', 'rsr_id', 'year', 'status'])

In [153]:
print(df.set_index(['funder_name', 'grant_id', 'rsr_id', 'year', 'status']).index.is_unique)
print(df.shape)
df.describe(include='all')

True
(243386, 20)


,funder_name,grant_id,rsr_id,end_date,funding_amount,start_date,funding_len,nb_rsrs,start_year,status,year,avg_fund_len,avg_team_size,fund_amt,nb_grnts,citations_per_pub,nb_pubs,team_size,nb_collab_countries,nb_collabs
count,243386,243386,243386,242858,2.181410e+05,243386,242858.000000,243386.000000,243386.000000,243386,243386.000000,69034.000000,69334.000000,6.348500e+04,243386.000000,25700.000000,243386.000000,25700.000000,243386.000000,243386.000000
unique,6,16926,14005,1081,NaN,1171,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,National Cancer Institute,grant.2438876,ur.01117731572.33,2017-01-01 00:00:00,NaN,2009-01-01 00:00:00,NaN,NaN,NaN,pre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,161502,495,4224,9108,NaN,17743,NaN,NaN,NaN,110630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,2007-01-31 00:00:00,NaN,2007-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2024-10-31 00:00:00,NaN,2012-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,2.336625e+06,NaN,4.228358,2.274790,2009.447483,NaN,2009.447483,3.871640,2.565341,2.023066e+06,1.125784,73.874731,0.734311,32.175308,0.487949,6.811686
std,NaN,NaN,NaN,NaN,9.191981e+06,NaN,2.655319,3.368468,1.661283,NaN,3.572101,2.404287,3.433619,4.693248e+06,15.541042,215.682098,4.405307,139.369226,2.537770,44.181902
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,0.002740,1.000000,2007.000000,NaN,2002.000000,0.002740,1.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,3.186080e+05,NaN,2.002740,1.000000,2008.000000,NaN,2007.000000,2.002740,1.000000,3.537750e+05,0.000000,12.000000,0.000000,7.750000,0.000000,0.000000


In [154]:
df.to_csv('../data/regression_dataset.csv', index=False)

## Sandbox